In [2]:
import pandas as pd
import nltk

In [8]:
data = pd.read_csv(r"C:\Users\user\Desktop\DATA BACK UP\Data_sdsouza33\LAMBTON\SEM 3\NLP\project\pse_isr_reddit_comments.csv")

In [9]:
data.head()

,comment_id,score,self_text,subreddit,created_time
0,k7kxio7,1,k,CombatFootage,2023-11-02 23:49:37+00:00
1,k7kxho7,1,What is he firing? Looks home made? And what i...,CombatFootage,2023-11-02 23:49:25+00:00
2,k7kxhem,1,I mean you did run away from the question firs...,CombatFootage,2023-11-02 23:49:22+00:00
3,k7kxgzg,2,There is a difference between footage showing ...,CombatFootage,2023-11-02 23:49:18+00:00
4,k7kxgne,1,Not feeding you anymore.,CombatFootage,2023-11-02 23:49:14+00:00


In [10]:
data.describe()

,score
count,513557.000000
mean,25.441121
std,164.527875
min,-1372.000000
25%,1.000000
50%,2.000000
75%,9.000000
max,16463.000000


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513557 entries, 0 to 513556
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   comment_id    513557 non-null  object
 1   score         513557 non-null  int64 
 2   self_text     513555 non-null  object
 3   subreddit     513557 non-null  object
 4   created_time  513557 non-null  object
dtypes: int64(1), object(4)
memory usage: 19.6+ MB
